<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#라이브러리-로드" data-toc-modified-id="라이브러리-로드-1">라이브러리 로드</a></span></li><li><span><a href="#데이터-준비" data-toc-modified-id="데이터-준비-2">데이터 준비</a></span><ul class="toc-item"><li><span><a href="#데이터셋-저장" data-toc-modified-id="데이터셋-저장-2.1">데이터셋 저장</a></span></li><li><span><a href="#폴더-생성-및-압축-해제" data-toc-modified-id="폴더-생성-및-압축-해제-2.2">폴더 생성 및 압축 해제</a></span></li><li><span><a href="#이미지-resize-함수-정의" data-toc-modified-id="이미지-resize-함수-정의-2.3">이미지 resize 함수 정의</a></span></li><li><span><a href="#Train-dataset-리사이즈" data-toc-modified-id="Train-dataset-리사이즈-2.4">Train dataset 리사이즈</a></span></li><li><span><a href="#Test-dataset-리사이즈" data-toc-modified-id="Test-dataset-리사이즈-2.5">Test dataset 리사이즈</a></span></li><li><span><a href="#tf.dataset-만들기" data-toc-modified-id="tf.dataset-만들기-2.6">tf.dataset 만들기</a></span></li></ul></li><li><span><a href="#모델-정의" data-toc-modified-id="모델-정의-3">모델 정의</a></span><ul class="toc-item"><li><span><a href="#모델-훈련" data-toc-modified-id="모델-훈련-3.1">모델 훈련</a></span></li><li><span><a href="#모델-평가-및-결론" data-toc-modified-id="모델-평가-및-결론-3.2">모델 평가 및 결론</a></span></li></ul></li><li><span><a href="#모델-개선" data-toc-modified-id="모델-개선-4">모델 개선</a></span><ul class="toc-item"><li><span><a href="#새로운-데이터셋-생성" data-toc-modified-id="새로운-데이터셋-생성-4.1">새로운 데이터셋 생성</a></span></li><li><span><a href="#transparent-and-resize" data-toc-modified-id="transparent-and-resize-4.2">transparent and resize</a></span><ul class="toc-item"><li><span><a href="#이미지-투명하게-만들기" data-toc-modified-id="이미지-투명하게-만들기-4.2.1">이미지 투명하게 만들기</a></span></li><li><span><a href="#배경-이미지-png로-저장하기" data-toc-modified-id="배경-이미지-png로-저장하기-4.2.2">배경 이미지 png로 저장하기</a></span></li><li><span><a href="#이미지-경로-저장" data-toc-modified-id="이미지-경로-저장-4.2.3">이미지 경로 저장</a></span></li><li><span><a href="#resize,-rotate,-flip-함수-작성" data-toc-modified-id="resize,-rotate,-flip-함수-작성-4.2.4">resize, rotate, flip 함수 작성</a></span></li></ul></li><li><span><a href="#tf.dataset-만들기" data-toc-modified-id="tf.dataset-만들기-4.3">tf.dataset 만들기</a></span><ul class="toc-item"><li><span><a href="#만들어진-데이터셋-예시" data-toc-modified-id="만들어진-데이터셋-예시-4.3.1">만들어진 데이터셋 예시</a></span></li><li><span><a href="#Train,-Valid-set-나누기" data-toc-modified-id="Train,-Valid-set-나누기-4.3.2">Train, Valid set 나누기</a></span></li></ul></li><li><span><a href="#모델-정의" data-toc-modified-id="모델-정의-4.4">모델 정의</a></span><ul class="toc-item"><li><span><a href="#Resnet101-Pretrained-모델-사용" data-toc-modified-id="Resnet101-Pretrained-모델-사용-4.4.1">Resnet101 Pretrained 모델 사용</a></span></li></ul></li><li><span><a href="#결론" data-toc-modified-id="결론-4.5">결론</a></span></li></ul></li></ul></div>

# 라이브러리 로드

In [5]:
import glob
import os
import random 
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import Image as display


SEED = 42 
random.seed(SEED) 
tf.random.set_seed(SEED) 
np.random.seed(SEED)

AUTOTUNE = tf.data.AUTOTUNE
cwd = Path(os.getcwd())

In [6]:
classes = ['scissor', 'rock', 'paper']

# 데이터 준비

https://teachablemachine.withgoogle.com/
  
위 사이트에 들어가서 데이터를 준비하면 됩니다.  
예시 사진은 아래와 같습니다.

<img src="https://i.ibb.co/n7kYQNY/1.png" width="500"/>

## 데이터셋 저장
`data/01` 경로 안에 paper.zip, scissor.zip, rock.zip 파일을 준비합니다.

<img src="https://i.ibb.co/gtNNgQy/2.png" width="500"/>

## 폴더 생성 및 압축 해제

In [ ]:
!mkdir data/01/paper
!mkdir data/01/scissor
!mkdir data/01/rock

!unzip data/01/paper.zip -d data/01/paper
!unzip data/01/scissor.zip -d data/01/scissor
!unzip data/01/rock.zip -d data/01/rock

## 이미지 resize 함수 정의

In [ ]:
def resize_images(img_path, save_path, target_size=(28,28)):
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    images= list(img_path.glob("*.jpg"))
    
    print(len(images), " images to be resized.")

    for img in images:
        filename = os.path.basename(img)
        old_img=Image.open(img)
        new_img=old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(save_path / filename, "JPEG")

    print(len(images), " images resized.")   


## Train dataset 리사이즈

In [ ]:
for class_ in classes:
    origin_img_path = cwd / 'data/01' / class_
    save_path = cwd / 'data/01/resize' / class_
    resize_images(origin_img_path, save_path)
    print(class_, '작업완료', '\n')

## Test dataset 리사이즈

In [ ]:
for class_ in classes:
    cwd = Path(os.getcwd())
    origin_img_path = cwd / 'data/01/test' / class_
    save_path = cwd / 'data/01/resize/test' / class_
    resize_images(origin_img_path, save_path)
    print(class_, '작업완료', '\n')

## tf.dataset 만들기

In [7]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    one_hot = parts[-2] == classes
    return tf.argmax(one_hot)

def decode_img(img):
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label


In [8]:
paths = []
for class_ in classes:
    data_path = cwd / 'data/01/resize' / class_
    paths += list(data_path.glob("*.jpg"))
paths = [str(_) for _ in paths]
list_ds = tf.data.Dataset.list_files(paths, shuffle=False)
train_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE).shuffle(300, seed=SEED).batch(32)

In [9]:
test_paths = []
for class_ in classes:
    data_path = cwd / 'data/01/resize/test' / class_
    test_paths += list(data_path.glob("*.jpg"))
test_paths = [str(_) for _ in test_paths]

test_ds = tf.data.Dataset.list_files(test_paths, shuffle=False)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE).batch(32)

# 모델 정의

In [10]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 1203      
Total params: 2,595
Trainable params: 2,595
Non-trainable params: 0
______________________________________________________

In [11]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

## 모델 훈련

In [12]:
model.fit(train_ds, epochs=10)

Epoch 1/10
10/10 [==============================] - 1s 6ms/step - loss: 32.5393 - accuracy: 0.3600
Epoch 2/10
10/10 [==============================] - 0s 2ms/step - loss: 13.1165 - accuracy: 0.4200
Epoch 3/10
10/10 [==============================] - 0s 2ms/step - loss: 6.8438 - accuracy: 0.5600
Epoch 4/10
10/10 [==============================] - 0s 2ms/step - loss: 4.1239 - accuracy: 0.6333
Epoch 5/10
10/10 [==============================] - 0s 2ms/step - loss: 2.3330 - accuracy: 0.7567
Epoch 6/10
10/10 [==============================] - 0s 2ms/step - loss: 2.1717 - accuracy: 0.7600
Epoch 7/10
10/10 [==============================] - 0s 2ms/step - loss: 0.8544 - accuracy: 0.8433
Epoch 8/10
10/10 [==============================] - 0s 2ms/step - loss: 0.4897 - accuracy: 0.8867
Epoch 9/10
10/10 [==============================] - 0s 2ms/step - loss: 0.3943 - accuracy: 0.9200
Epoch 10/10
10/10 [==============================] - 0s 2ms/step - loss: 0.2440 - accuracy: 0.9367


In [13]:
model.evaluate(test_ds)

1/1 [==============================] - 0s 112ms/step - loss: 10.1377 - accuracy: 0.2500


[10.137654304504395, 0.25]

## 모델 평가 및 결론

결과
- train set 정확도는 99.67% 를 달성했지만, test set 정확도는 25%에 불과하다.

이유
- Overfitting이 발생했다.
- 데이터가 충분하지 않다.
- 각 데이터가 모두 비슷하다.

=> 새로운 손 데이터 셋을 찾아보자.  
=> 그리고 각 이미지 샘플의 배경을 다르게 해보자.

# 모델 개선

<img src="https://i.ibb.co/vLqmP8z/4.png" width="500"/>

- **기본 아이디어**  
1. 데이터  
위 사진과 같이, 손 그림의 흰 영역을 투명으로 처리한 후, 배경이미지 위에 올려 다양한 배경 위에 손이 존재하도록 한다.  
 이 때 손은 회전과 크기조절을 랜덤하게 적용한다.  
 배경 또한 300장의 이미지를 랜덤하게 적용한다.

  
2. 모델
pretrained model을 사용하자

## 새로운 데이터셋 생성

아래 사이트에서 가위바위보 손모양 데이터셋 다운로드 받기  
https://laurencemoroney.com/datasets.html#rock-paper-scissors-dataset  
**압축 파일을 풀고, scissors 폴더 이름을 scissor 로 바꿔주세요**

<img src="https://i.ibb.co/z4S9KSS/2021-09-28-01-55-14.png" width="400"/>

배경으로 쓸 사진 다운로드 받기  
https://storage.googleapis.com/openimages/web/download.html

<img src="https://storage.googleapis.com/openimages/web/images/fiftyone.png" width="500"/>

**data/01 폴더 안에 hand 폴더를 만들고, 위에서 받은 손모양 이미지들을 준비해주세요.  
data/01 폴더 안에 background 폴더를 만들고, 위에서 받은 배경 이미지들을 준비해주세요.**

<img src="https://i.ibb.co/VCw4HDW/7.png" width="300"/>

## transparent and resize

### 이미지 투명하게 만들기

In [14]:
from PIL import Image

def make_transparent(img_path, save_path):
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    images= list(img_path.glob("*.png"))
    for img in images:
        filename = os.path.basename(img)
        img = Image.open(img)
        img = img.convert("RGBA")
        datas = img.getdata()
        newData = []
        for item in datas:
            if 225 <= item[0] <= 255 and 225 <= item[1] <= 255 and 225 <= item[2] <= 255:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)
        img.putdata(newData)
        img.save(save_path / filename, "PNG")

In [ ]:
for class_ in classes:
    img_path = cwd / 'data/01/hand' / class_
    save_path = cwd / 'data/01/resize/hand' / class_
    make_transparent(img_path, save_path)

### 배경 이미지 png로 저장하기

In [15]:
def save_png(img_path, save_path, target_size=(224, 224)):
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    
    images= list(img_path.glob("*.jpg"))
    
    print(len(images), " images to be resized.")

    for img in images:
        filename = os.path.basename(img) + '.png'
        img = Image.open(img)
        width, height = img.size
        img = img.resize(target_size)
        img = img.convert("RGBA")
        img.save(save_path / filename, "PNG")

    print(len(images), " images resized.")   


In [ ]:
img_path = cwd / 'data/01/background'
save_path = cwd / 'data/01/resize/background'
save_png(img_path, save_path)

### 이미지 경로 저장

In [16]:
background_paths = cwd / 'data/01/resize/background'
background_paths =  list(background_paths.glob("*.png"))
background_paths = [str(_) for _ in background_paths]

In [17]:
hand_paths = []
for class_ in classes:
    data_path = cwd / 'data/01/resize/hand' / class_
    hand_paths += list(data_path.glob("*.png"))
hand_paths = [str(_) for _ in hand_paths]

### resize, rotate, flip 함수 작성

In [18]:
def resize(img, prob, ratio):
    w, h = img.size
    if np.random.rand() < prob:
        ratio = max(ratio, np.random.rand())
        w = int(w*ratio)
        h = int(h*ratio)
        img = img.resize((w, h))
    return img


def rotate(img, prob):
    if np.random.rand() < prob:
        theta = int(np.random.rand() * 90)
        img = img.rotate(angle=theta)
    return img


def flip(img, prob):
    if np.random.rand() < prob:
        if np.random.rand() < 0.5:
            img = np.flip(img, 0)
        else:
            img = np.flip(img, 1)
    return img


def image_merge(back, front):
    parts = tf.strings.split(front, os.path.sep)
    one_hot = parts[-2] == classes
    label = tf.argmax(one_hot)
    label = tf.cast(label, tf.int8)
    
    back = back.numpy()
    front = front.numpy()
    background = Image.open(back)
    foreground = Image.open(front)
    
    foreground = resize(foreground, 0.5, 0.5)
    foreground = rotate(foreground, 0.5)
    
    
    background.paste(foreground, (0, 0), foreground)
    background = background.convert("RGB")
    background = np.array(background)
    background = flip(background, 0.5)
    
    
    return background, label


## tf.dataset 만들기

In [19]:
train_ds_hand = tf.data.Dataset.list_files(hand_paths, seed=SEED).shuffle(300, seed=SEED)
train_ds_background = tf.data.Dataset.list_files(background_paths, seed=SEED).shuffle(300, seed=SEED)
train_ds = tf.data.Dataset.zip((train_ds_background, train_ds_hand))

In [20]:
def tf_img_merge(back, front):
    image, label = tf.py_function(image_merge, [back, front], [tf.int16, tf.int8])
    image.set_shape((224,224,3))
    label.set_shape(())
    return image, label

In [21]:
img_merge_ds = train_ds.map(tf_img_merge)
img_merge_ds = img_merge_ds.batch(32)

In [22]:
# for a in img_merge_ds.take(1):
#     plt.imshow(a[0][0].numpy())
#     plt.show()

### 만들어진 데이터셋 예시

<img src="https://i.ibb.co/ZSYPS7y/6.png" width="500"/>

### Train, Valid set 나누기

In [23]:
def is_test(x, y):
    return x % 4 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

test_dataset = img_merge_ds.enumerate().filter(is_test).map(recover)
train_dataset = img_merge_ds.enumerate().filter(is_train).map(recover)

## 모델 정의

### Resnet101 Pretrained 모델 사용

In [24]:
base_model = keras.applications.ResNet101(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [25]:
# freeze 한 후, 모델 훈련

base_model.trainable = False

In [26]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(3, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet101 (Functional)       (None, 7, 7, 2048)        42658176  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 6147      
Total params: 42,664,323
Trainable params: 6,147
Non-trainable params: 42,658,176
_________________________________________________________________


In [28]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

Epoch 1/10
7/7 [==============================] - 11s 1s/step - loss: 1.8599 - accuracy: 0.3922 - val_loss: 1.1374 - val_accuracy: 0.4688
Epoch 2/10
7/7 [==============================] - 6s 823ms/step - loss: 1.5432 - accuracy: 0.4363 - val_loss: 1.1715 - val_accuracy: 0.4583
Epoch 3/10
7/7 [==============================] - 5s 774ms/step - loss: 1.5462 - accuracy: 0.4314 - val_loss: 1.0385 - val_accuracy: 0.4583
Epoch 4/10
7/7 [==============================] - 5s 794ms/step - loss: 1.3413 - accuracy: 0.5392 - val_loss: 0.7795 - val_accuracy: 0.6562
Epoch 5/10
7/7 [==============================] - 5s 817ms/step - loss: 1.3267 - accuracy: 0.5196 - val_loss: 0.8675 - val_accuracy: 0.5938
Epoch 6/10
7/7 [==============================] - 5s 791ms/step - loss: 1.1829 - accuracy: 0.5735 - val_loss: 0.6421 - val_accuracy: 0.7812
Epoch 7/10
7/7 [==============================] - 5s 808ms/step - loss: 1.1787 - accuracy: 0.5637 - val_loss: 0.5709 - val_accuracy: 0.7500
Epoch 8/10
7/7 [======

In [29]:
# unfreeze 한 후, 모델 훈련

base_model.trainable = True

model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset, epochs=20, validation_data=test_dataset)

Epoch 1/20
7/7 [==============================] - 15s 1s/step - loss: 0.7884 - accuracy: 0.6814 - val_loss: 0.4796 - val_accuracy: 0.8021
Epoch 2/20
7/7 [==============================] - 7s 988ms/step - loss: 0.7173 - accuracy: 0.7059 - val_loss: 0.4168 - val_accuracy: 0.8542
Epoch 3/20
7/7 [==============================] - 7s 984ms/step - loss: 0.5161 - accuracy: 0.7696 - val_loss: 0.3277 - val_accuracy: 0.8958
Epoch 4/20
7/7 [==============================] - 7s 971ms/step - loss: 0.4628 - accuracy: 0.7892 - val_loss: 0.3187 - val_accuracy: 0.8438
Epoch 5/20
7/7 [==============================] - 7s 974ms/step - loss: 0.3941 - accuracy: 0.8529 - val_loss: 0.2719 - val_accuracy: 0.8854
Epoch 6/20
7/7 [==============================] - 7s 1s/step - loss: 0.3340 - accuracy: 0.8873 - val_loss: 0.1506 - val_accuracy: 0.9688
Epoch 7/20
7/7 [==============================] - 7s 1s/step - loss: 0.2925 - accuracy: 0.8873 - val_loss: 0.1166 - val_accuracy: 0.9688
Epoch 8/20
7/7 [============

In [30]:
test_paths = []
for class_ in classes:
    data_path = cwd / 'data/01/test' / class_
    test_paths += list(data_path.glob("*.jpg"))
test_paths = [str(_) for _ in test_paths]

test_ds = tf.data.Dataset.list_files(test_paths, shuffle=False)
test_ds = test_ds.map(process_path).batch(12)

In [31]:
model.evaluate(test_ds)

1/1 [==============================] - 1s 1s/step - loss: 0.6892 - accuracy: 0.9167


[0.6891592144966125, 0.9166666865348816]

In [32]:
pred = model.predict(test_ds)
np.argmax(pred, axis = -1)

array([2, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0])

In [33]:
y = [j for i, j in test_ds]
y

[<tf.Tensor: shape=(12,), dtype=int64, numpy=array([2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0])>]

## 결론
정확도 25%에서 91.66% 로 상승 

아래 사진을 주먹으로 인식하는 에러 존재

<img src="https://i.ibb.co/grHTH2r/2021-09-30-19-51-39.png" width=150/>